In [1]:
import pandas as pd 
import numpy as np

In [2]:
data = pd.read_csv("DataFolder/Final_Copy_15_Oct.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'DataFolder/Final_Copy_15_Oct.csv'

In [5]:
import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import DatasetDict, load_dataset
import pandas as pd

# Assuming you have your dataset in two separate files or combined in a DataFrame
# # Replace 'path_to_your_data.csv' with the actual paths to your dataset filaes
# dataset = load_dataset("csv", data_files={"train": "DataFolder/Final_Copy_15_Oct.csv"})
# train_test_split = dataset["train"].train_test_split(test_size=0.2)  
# dataset = DatasetDict({
#     "train": train_test_split["train"],
#     "validation": train_test_split["test"]
# })

# First load the dataset
dataset = load_dataset("csv", data_files={"train": "DataFolder/Final_Copy_15_Oct.csv"})

# First split: separate out test set (20% of total data)
first_split = dataset["train"].train_test_split(test_size=0.1)
train_valid_data = first_split["train"]
test_data = first_split["test"]

# Second split: split remaining data into train and validation (80/20 split of remaining data)
train_valid_split = train_valid_data.train_test_split(test_size=0.1)

# Create the final DatasetDict with all three splits
dataset = DatasetDict({
    "train": train_valid_split["train"],
    "validation": train_valid_split["test"],
    "test": test_data
})

# Save the test set as CSV
test_df = dataset["test"].to_pandas()
test_df.to_csv("DataFolder/test_set.csv", index=False)


model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    # Tokenize input and target texts separately and add them to the model_inputs dictionary
    tokenized_inputs = tokenizer(examples["dzo"], padding="max_length", truncation=True, max_length=128)
    tokenized_targets = tokenizer(examples["eng"], padding="max_length", truncation=True, max_length=128)
    
    # Ensure the "labels" field contains the tokenized targets
    model_inputs = {k: v for k, v in tokenized_inputs.items()}
    model_inputs["labels"] = tokenized_targets["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="./final_dzo_eng_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=3,
    predict_with_generate=True
)

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()
model.save_pretrained("dzo_to_eng_nllb_finetuned")
tokenizer.save_pretrained("dzo_to_eng_nllb_finetuned")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


FileNotFoundError: Unable to find '/home/nmt/Desktop/NMT/src/DataFolder/Final_Copy_15_Oct.csv'